# Tutorial 14: Class Parameters

This tutorial walks you through the process of using different parameters classes. Understanding how these classes work is fundamental in designing your own scenarios with the use of Flow. The classes featured in this tutorial are the `InitialConfig`, `SUMOCarFollowingParams`, `EnvParams` and the `SUMOParams`. Each of these classes are detailed in their corresponding section in this tutorial.

This tutorial uses the classical Sugiyama experiment involving a loop (ring road) with single lane and a fixed number of vehicles on it, similar to Tutorial 01. This one though focuses in the use of the parameters in the classes.

### This tutorial is organized as follows:

* `InitialConfig`
* `SUMOCarFollowingParams`
* `EnvParams`
* `SUMOParams`


## 1. Using Param class: Initial Configuration

`InitialConfig` specifies parameters that affect the positioning of vehicle in the network at the start of a simulation. These parameters can be used to limit the edges and number of lanes vehicles originally occupy, and provide a means of adding randomness to the starting positions of vehicles. In order to introduce a small initial disturbance to the system of vehicles in the network, we set the `perturbation` term in `InitialConfig` to 1m.

    def __init__(self,
                 shuffle=False,
                 spacing="uniform",
                 min_gap=0,
                 perturbation=0.0,
                 x0=0,
                 bunching=0,
                 lanes_distribution=float("inf"),
                 edges_distribution="all",
                 additional_params=None):
        
        self.shuffle = shuffle
        self.spacing = spacing
        self.min_gap = min_gap
        self.perturbation = perturbation
        self.x0 = x0
        self.bunching = bunching
        self.lanes_distribution = lanes_distribution
        self.edges_distribution = edges_distribution
        self.additional_params = additional_params or dict()

    
    """Instantiate InitialConfig.

        These parameters that affect the positioning of vehicle in the
        network at the start of a rollout. By default, vehicles are uniformly
        distributed in the network.

        Attributes
        ----------
        shuffle: bool, optional
            specifies whether the ordering of vehicles in the Vehicles class
            should be shuffled upon initialization.
        spacing: str, optional
            specifies the positioning of vehicles in the network relative to
            one another. May be one of: "uniform", "random", or "custom".
            Default is "uniform".
        min_gap: float, optional
            minimum gap between two vehicles upon initialization, in meters.
            Default is 0 m.
        x0: float, optional
            position of the first vehicle to be placed in the network
        perturbation: float, optional
            standard deviation used to perturb vehicles from their uniform
            position, in meters. Default is 0 m.
        bunching: float, optional
            reduces the portion of the network that should be filled with
            vehicles by this amount.
        lanes_distribution: int, optional
            number of lanes vehicles should be dispersed into. If the value is
            greater than the total number of lanes on an edge, vehicles are
            spread across all lanes.
        edges_distribution: list <str>, optional
            list of edges vehicles may be placed on initialization, default is
            all lanes (stated as "all")
        additional_params: dict, optional
            some other network-specific params
        """
        
All sections in this tutorial should consist of blocks of text describing a specific method or set of methods in Flow, following by a code snippet demonstrating the described methods being used.

For example, let us say we want to show how to add two numbers in python. We might begin by introducing the numbers and methods we will use to do the addition:

In [5]:
# This part initializes the Sugiyama experiment. 

from flow.controllers import IDMController, ContinuousRouter
from flow.core.experiment import Experiment
from flow.core.params import SumoParams, EnvParams, \
    InitialConfig, NetParams
from flow.core.params import VehicleParams
from flow.envs.loop.loop_accel import AccelEnv, ADDITIONAL_ENV_PARAMS
from flow.scenarios.loop import LoopScenario, ADDITIONAL_NET_PARAMS


# def sugiyama_example(render=None):
#     """
#     Perform a simulation of vehicles on a ring road.

#     Parameters
#     ----------
#     render : bool, optional
#         specifies whether to use the gui during execution

#     Returns
#     -------
#     exp: flow.core.experiment.Experiment
#         A non-rl experiment demonstrating the performance of human-driven
#         vehicles on a ring road.
#     """
sim_params = SumoParams(sim_step=0.1, render=True)

if render is not None:
       sim_params.render = render

vehicles = VehicleParams()
vehicles.add(
    veh_id="idm",
    acceleration_controller=(IDMController, {}),
    routing_controller=(ContinuousRouter, {}),
    num_vehicles=22)
        
        
env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)

additional_net_params = ADDITIONAL_NET_PARAMS.copy()
net_params = NetParams(additional_params=additional_net_params)

ModuleNotFoundError: No module named 'flow'

## This is the section where you should be able to modify the parameters within the `InitialConfig` class.

In [2]:
    initial_config = InitialConfig(bunching=20)

NameError: name 'InitialConfig' is not defined

In [ ]:
# This completes the scenario and should be able to run the simulation in the SUMO GUI.

    scenario = LoopScenario(
        name="sugiyama",
        vehicles=vehicles,
        net_params=net_params,
        initial_config=initial_config)

    env = AccelEnv(env_params, sim_params, scenario)

    return Experiment(env)


if __name__ == "__main__":
    # import the experiment variable
    exp = sugiyama_example()

    # run for a set number of rollouts / time steps
    exp.run(1, 1500)

Then we could show the functionality of the methods as follows:

In [ ]:
add(a, b)

Whenever possible, sections should also be broken up into smaller subsections to help the reader more quickly identify which portion of the tutorial discusses which topic. This may be helpful to readers who are just interested in certain concept, e.g. just to find out how a specific parameter works. An example of this separation of subsections can be seen in Section 2.

## 2. SumoCarFollowingParams

"""Parameters for sumo-controlled acceleration behavior."""
   
   
   def __init__(
            self,
            speed_mode='right_of_way',
            accel=1.0,
            decel=1.5,
            sigma=0.5,
            tau=1.0,  # past 1 at sim_step=0.1 you no longer see waves
            min_gap=2.5,
            max_speed=30,
            speed_factor=1.0,
            speed_dev=0.1,
            impatience=0.5,
            car_follow_model="IDM",
            **kwargs):

    """Instantiate SumoCarFollowingParams.

        Attributes
        ----------
        speed_mode : str or int, optional
            may be one of the following:

             * "right_of_way" (default): respect safe speed, right of way and
               brake hard at red lights if needed. DOES NOT respect
               max accel and decel which enables emergency stopping.
               Necessary to prevent custom models from crashing
             * "no_collide": Human and RL cars are preventing from reaching
               speeds that may cause crashes (also serves as a failsafe). Note:
               this may lead to collisions in complex networks
             * "aggressive": Human and RL cars are not limited by sumo with
               regard to their accelerations, and can crash longitudinally
             * "all_checks": all sumo safety checks are activated
             * int values may be used to define custom speed mode for the given
               vehicles, specified at:
               http://sumo.dlr.de/wiki/TraCI/Change_Vehicle_State#speed_mode_.280xb3.29

        accel: float
            see Note
        decel: float
            see Note
        sigma: float
            see Note
        tau: float
            see Note
        min_gap: float
            see minGap Note
        max_speed: float
            see maxSpeed Note
        speed_factor: float
            see speedFactor Note
        speed_dev: float
            see speedDev in Note
        impatience: float
            see Note
        car_follow_model: str
            see carFollowModel in Note
        kwargs: dict
            used to handle deprecations
            
            
        Note
        ----
        For a description of all params, see:
        http://sumo.dlr.de/wiki/Definition_of_Vehicles,_Vehicle_Types,_and_Routes

        """
           
        # check for deprecations (minGap)
        if "minGap" in kwargs:
            deprecation_warning(self, "minGap", "min_gap")
            min_gap = kwargs["minGap"]

        # check for deprecations (maxSpeed)
        if "maxSpeed" in kwargs:
            deprecation_warning(self, "maxSpeed", "max_speed")
            max_speed = kwargs["maxSpeed"]

        # check for deprecations (speedFactor)
        if "speedFactor" in kwargs:
            deprecation_warning(self, "speedFactor", "speed_factor")
            speed_factor = kwargs["speedFactor"]

        # check for deprecations (speedDev)
        if "speedDev" in kwargs:
            deprecation_warning(self, "speedDev", "speed_dev")
            speed_dev = kwargs["speedDev"]

        # check for deprecations (carFollowModel)
        if "carFollowModel" in kwargs:
            deprecation_warning(self, "carFollowModel", "car_follow_model")
            car_follow_model = kwargs["carFollowModel"]

        # create a controller_params dict with all the specified parameters
        self.controller_params = {
            "accel": accel,
            "decel": decel,
            "sigma": sigma,
            "tau": tau,
            "minGap": min_gap,
            "maxSpeed": max_speed,
            "speedFactor": speed_factor,
            "speedDev": speed_dev,
            "impatience": impatience,
            "carFollowModel": car_follow_model,
        }

        # adjust the speed mode value
        if isinstance(speed_mode, str) and speed_mode in SPEED_MODES:
            speed_mode = SPEED_MODES[speed_mode]
        elif not (isinstance(speed_mode, int)
                  or isinstance(speed_mode, float)):
            logging.error("Setting speed mode of to default.")
            speed_mode = SPEED_MODES["no_collide"]

        self.speed_mode = speed_mode


### 2.1 Introduce and Instantiate the Class

We begin by defining the class with its `__init__` method, and import any necessary modules. Make sure there are enough comments in the code to make it s much self-explanatory as possible.

In [ ]:
import numpy as np


class Adder(object):

    def __init__(self, numbers):
        """Instantiate the Adder class.

        Parameters
        ----------
        numbers : array_like
            the numbers that should be added together.
        """
        self.numbers = numbers

### 2.2 Include (Sequentially) New Methods

Next, we add a new method to the class. We do this by recreating the class and having it inherit its previous properties. As an example of this, let us consider including a `run` method to the `Adder` class than adds the numbers that is provided to the class during instantiation.

In [ ]:
class Adder(Adder):  # continuing from the previous definition of Adder

    def run(self):
        return np.sum(self.numbers)

### 2.3 Demonstrate Functionality of the Class

Finally, we can demonstrate the functionality of the class, through testing and validating the class, as seen in the code snippet below.

In [ ]:
adder = Adder([1, 2, 3])

print("The sum of the values is:", adder.run())

## 3. EnvParams

    """Environment and experiment-specific parameters.

    This includes specifying the bounds of the action space and relevant
    coefficients to the reward function, as well as specifying how the
    positions of vehicles are modified in between rollouts.
    """
    
    def __init__(self,
                 additional_params=None,
                 horizon=500,
                 warmup_steps=0,
                 sims_per_step=1,
                 evaluate=False):
                 
    """Instantiate EnvParams.

        Attributes
        ----------
            additional_params: dict, optional
                Specify additional environment params for a specific
                environment configuration
            horizon: int, optional
                number of steps per rollouts
            warmup_steps: int, optional
                number of steps performed before the initialization of training
                during a rollout. These warmup steps are not added as steps
                into training, and the actions of rl agents during these steps
                are dictated by sumo. Defaults to zero
            sims_per_step: int, optional
                number of sumo simulation steps performed in any given rollout
                step. RL agents perform the same action for the duration of
                these simulation steps.
            evaluate: bool, optional
                flag indicating that the evaluation reward should be used
                so the evaluation reward should be used rather than the
                normal reward

        """
        
        
        self.additional_params = \
            additional_params if additional_params is not None else {}
        self.horizon = horizon
        self.warmup_steps = warmup_steps
        self.sims_per_step = sims_per_step
        self.evaluate = evaluate

        def get_additional_param(self, key):
        """Return a variable from additional_params."""
        return self.additional_params[key]
        
        

## 4. Adding the Changes to the README and Website

Once you have completed your tutorial, you must include your new tutorial in all relevant descriptors of Flow's tutorials. This include adding it to both README and the Flow Website.

### 3.1 README

For one, begin by adding the new tutorial to the README.md file located in the tutorials/ directory (see the figure below). This should be included in your Pull Request (PR) whenever creating a new tutorial.

<img src="img/tutorials_readme.png">

You just need to add your tutorial with the correct number and title under the last tutorial in the README.md:

`
**Tutorial XX:** Name of your tutorial.
`

### 3.2 Website

Next, you need to inform the Flow web designer to add your tutorial to the Flow website:

<img src="img/tutorials_website.png">

To do so, send the Title and the Github link to your tutorial to the Flow web designer.